# Librerias

In [1]:
import pickle
import pandas as pd
import numpy as np
import warnings
import re
from unidecode import unidecode
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup

warnings.filterwarnings('ignore')

# Mis funciones

In [2]:
import import_ipynb
from mis_funciones import *

importing Jupyter notebook from mis_funciones.ipynb


# Cargamos datos

In [6]:
# Volvemos a cargar el archivo de filtrado
df_beer = pd.read_csv("../01_data/filtro_beer.csv", delimiter=",", header=0)
df_beer

,url,local,nombre_show,n_opiniones,num_paginas
0,/entradas/ignatius-farray-presenta-nuevo-show-...,Beer Station - Sala 1,Ignatius Farray presenta nuevo show STAND-UP C...,571,58
1,/entradas/comedy-nights-monologos-en-beer-stat...,Beer Station - Sala 1,"Comedy Nights, monólogos en Beer Station",837,84
2,/entradas/hardcore-patricia-galvan_e4826004/,Beer Station - Sala 1,Hardcore - Patricia Galván,163,17
3,/entradas/yo-nunca-show-de-raul-massana_e289258/,Beer Station - Sala 1,"YO NUNCA SHOW, de Raúl Massana",157,16
4,/entradas/pablo-meixe-quien-aguanta-esto_e4884...,Beer Station - Sala 1,Pablo Meixe - ¿Quién aguanta esto?,27,3
5,/entradas/la-bromerie-open-special-edition-en-...,Beer Station - Sala 1,"La Bromerie Open Special Edition, en Beer Station",53,6
6,/entradas/chochito-un-monologo-de-carmen-romer...,Beer Station - Sala 1,"Chochito, un monólogo de Carmen Romero",17,2
7,/entradas/isabel-rey-diva-de-barrio-un-show-de...,Beer Station - Sala 1,"Isabel Rey - Diva de barrio, un show de comedia",51,6
8,/entradas/ruben-faura-la-hora-del-psicologo_e4...,Beer Station - Sala 1,Rubén Faura - La hora del psicólogo,288,29
9,/entradas/luis-alvaro-el-espectaculo_e64297/,Beer Station - Sala 1,Luis Álvaro - El espectáculo,203,21


In [4]:
# Función para extraer comentarios de una página específica
def extraer_comentarios(url, nombre_show, nombre_local):
    """
    Esta función extrae comentarios de una página web y devuelve una lista de diccionarios con información detallada sobre cada comentario.
    
    Requiere las siguientes bibliotecas:
    - requests
    - BeautifulSoup (bs4)
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    comentarios = soup.select(".rws-results-review-item")
    lista_comentarios = []

    for comentario in comentarios:
        nota_total = comentario.select_one(".aui-ratingBadge--large").text
        fecha = comentario.select_one(".meta-info__date").text
        grupo_de = comentario.select_one(".meta-info__company .aui-badge-text").text

        aspectos = comentario.select(".rws-circle-chart-list__item")

        if aspectos:
            calidad = aspectos[0].select_one(".rws-circle-chart__value-container span").text
            puesta_en_escena = aspectos[1].select_one(".rws-circle-chart__value-container span").text
            interpretacion = aspectos[2].select_one(".rws-circle-chart__value-container span").text

            comentario_dict = {
                "fecha": fecha,
                "nota_total": nota_total,
                "grupo_de": grupo_de,
                "calidad_del_espectáculo": calidad,
                "puesta_en_escena": puesta_en_escena,
                "interpretación_artística": interpretacion,
                "nombre_show": nombre_show,  
                "nombre_local": nombre_local  
            }

            lista_comentarios.append(comentario_dict)

    return lista_comentarios

def convertir_fecha(fecha):
    meses = {
        "enero": "January",
        "febrero": "February",
        "marzo": "March",
        "abril": "April",
        "mayo": "May",
        "junio": "June",
        "julio": "July",
        "agosto": "August",
        "septiembre": "September",
        "octubre": "October",
        "noviembre": "November",
        "diciembre": "December"
    }
    partes = fecha.split()
    partes[1] = meses[unidecode(partes[1].lower())]
    fecha_nueva = " ".join(partes)
    return pd.to_datetime(fecha_nueva, format='%d %B %Y', errors='coerce')

# Continuamos extracción

Para ello, lo primero que hacemos es buscar donde nos hemos quedado:

In [8]:
# Encontrar el índice donde se quedó la extracción
indice_reanudacion = df_beer[df_beer['nombre_show'] == 'Pablo Meixe - ¿Quién aguanta esto?'].index[0]
indice_reanudacion

4

Reiniciamos:

In [10]:
# URL base de la página
base_url = "https://www.atrapalo.com"

# Inicializar lista para almacenar todos los comentarios
todos_los_comentarios = []

# Iterar a través de las filas de df_beer
for index, row in df_beer.iloc[indice_reanudacion:].iterrows():
    url = f"{base_url}{row['url']}"
    nombre_show = row['nombre_show']
    nombre_local = row['local']
    num_paginas = row['num_paginas']
    
    # Iterar a través de las páginas
    for i in range(1, num_paginas + 1):
        url_pagina = f"{url}page-{i}/"
        comentarios_pagina = extraer_comentarios(url_pagina, nombre_show, nombre_local)
        todos_los_comentarios.extend(comentarios_pagina)
    
    # Pasar a DataFrame los datos
    df_comentarios = pd.DataFrame(todos_los_comentarios)

    # Aplicar la función a la columna 'fecha'
    df_comentarios['fecha'] = df_comentarios['fecha'].apply(convertir_fecha)

    # Quitar 'sobre 10' de las columnas de calidad del espectáculo, puesta en escena e interpretación artística
    df_comentarios['calidad_del_espectáculo'] = df_comentarios['calidad_del_espectáculo'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['puesta_en_escena'] = df_comentarios['puesta_en_escena'].str.replace(r'\s+sobre\s+10', '', regex=True)
    df_comentarios['interpretación_artística'] = df_comentarios['interpretación_artística'].str.replace(r'\s+sobre\s+10', '', regex=True)


    # Remover caracteres especiales y espacios en blanco del nombre del show
    nombre_show = re.sub(r'[^\w\s]', '', nombre_show)

    # Reemplazar espacios en blanco con guiones bajos
    nombre_show = nombre_show.replace(' ', '_')

    # Guardar el DataFrame en un archivo CSV
    df_comentarios.to_csv(f"../01_data/comentarios_{nombre_show}.csv", index=False)